# Projeto 1 - Ciência dos Dados

Nome:Livia Brigido do Nascimento

Nome:Fabio Dotoli Ferreira Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [12]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/fabiodff/Desktop/P1Cdados2021_2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [13]:
filename = 'Airpods.xlsx'

In [14]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,pqp achei que eu tinha perdido a capinha do ai...,0.0
1,@taecgiri que airpods podre é esse que fica ca...,1.0
2,"@vanteanjo army bomb, um álbum do bts (qualque...",0.0
3,g perdu mes airpods pro 😐,0.0
4,não tô acreditando q esqueci meu airpods dentr...,0.0


In [15]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@stevenzmoon @klausbestofall @julietteftthali ...,0.0
1,a minha mãe comprou a uns dias airpods e não m...,0.0
2,"postar uma música de um aplicativo bom, lembre...",0.0
3,se aquele airpods n tivesse identificação do 9...,0.0
4,"sim, comprei airpods com o cartão do bozo hmm ...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

### Produto escolhido ###
Os Airpods, lançamento da Apple no ano de 2016, são pequenos fones com conexão via bluetooth, que vêm acompanhados do estojo (usado para recarregá-los) e de um cabo lightning (carregador).

- Os tweets considerados relevantes foram aqueles que citavam alguma caracteristica relacionada ao produto escolhido, como por exemplo a durabilidade, a facilidade em perder os fones, preço e críticas/elogios aos Airpods como um todo.

- Os tweets irrelevantes, por sua vez, são aqueles que citam o produto, porém de uma forma que não é possivel extrair um feedback significativo sobre satisfação ou qualidades (positivas ou negativas) como, por exemplo, uma frase citando alguns itens aleatórios (dentre eles, os Airpods).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**